# DigiKrawl: DigiKala Comment Crawler

## Prerequisites

The following code snippet import required third-party libraries:
+ `requests` for sending HTTP requests to DigiKala server
+ `bs4` for parsing and crawling HTTP responses
+ `pandas` for data manipulation
+ `datetime` for using time series data
+ `unicode` for decode Farsi characters
+ `jdatetime` for work with Jalali dates

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from unidecode import unidecode
import jdatetime 